# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [440]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
import random
import string
from pprint import pprint
from functions_variables import *

## Data Importing

#### - Testing files and paths

In [441]:
test_dir = '../data/AK_Juneau_0.json'
os.path.exists(test_dir)

True

In [442]:
# load one file first to see what type of data you're dealing with and what attributes it has
with open(test_dir) as f:
    data_json = json.load(f)

In [443]:
data_json['data'].keys()

dict_keys(['total', 'count', 'results'])

In [444]:
test_df = pd.json_normalize(data_json['data']['results'])
test_df.head(1)

,last_update_date,tags,permalink,status,list_date,open_houses,branding,list_price,property_id,photos,...,location.address.state_code,location.address.line,location.street_view_url,location.county.fips_code,location.county.name,primary_photo,source,products,location.address.coordinate,other_listings
0,2023-09-19T20:52:50Z,"[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,None,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...",...,AK,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN


#### - Data Overview

In [445]:
# Getting an overview of the data before combining all the files into one dataframe. This is a custom function in `functions_variables.py` --- It takes in a directory path.
overview = json_files_summary('../data')

Error processing file .gitkeep: Expecting value: line 1 column 1 (char 0)
AK_Juneau_0.json processed. 1/251 files processed.
AK_Juneau_1.json processed. 2/251 files processed.
AK_Juneau_2.json processed. 3/251 files processed.
AK_Juneau_3.json processed. 4/251 files processed.
AK_Juneau_4.json processed. 5/251 files processed.
AL_Montgomery_0.json processed. 6/251 files processed.
AL_Montgomery_1.json processed. 7/251 files processed.
AL_Montgomery_2.json processed. 8/251 files processed.
AL_Montgomery_3.json processed. 9/251 files processed.
AL_Montgomery_4.json processed. 10/251 files processed.
AR_LittleRock_0.json processed. 11/251 files processed.
AR_LittleRock_1.json processed. 12/251 files processed.
AR_LittleRock_2.json processed. 13/251 files processed.
AR_LittleRock_3.json processed. 14/251 files processed.
AR_LittleRock_4.json processed. 15/251 files processed.
AZ_Phoenix_0.json processed. 16/251 files processed.
AZ_Phoenix_1.json processed. 17/251 files processed.
AZ_Phoeni

In [446]:
overview
# After reviewing the data with Data Wrangler, I found that some files may be missing the 'results' key, or it is empty.

,file_name,file_path,rows_count,cols_count,cols_name
0,AK_Juneau_0.json,../data\AK_Juneau_0.json,8,64,"[last_update_date, tags, permalink, status, li..."
1,AK_Juneau_1.json,../data\AK_Juneau_1.json,7,63,"[primary_photo, last_update_date, source, tags..."
2,AK_Juneau_2.json,../data\AK_Juneau_2.json,6,63,"[primary_photo, last_update_date, source, tags..."
3,AK_Juneau_3.json,../data\AK_Juneau_3.json,5,63,"[primary_photo, last_update_date, source, tags..."
4,AK_Juneau_4.json,../data\AK_Juneau_4.json,4,63,"[primary_photo, last_update_date, source, tags..."
...,...,...,...,...,...
245,WY_Cheyenne_0.json,../data\WY_Cheyenne_0.json,1,0,[]
246,WY_Cheyenne_1.json,../data\WY_Cheyenne_1.json,1,0,[]
247,WY_Cheyenne_2.json,../data\WY_Cheyenne_2.json,1,0,[]
248,WY_Cheyenne_3.json,../data\WY_Cheyenne_3.json,1,0,[]


In [447]:
overview.shape

(250, 5)

In [448]:
overview

,file_name,file_path,rows_count,cols_count,cols_name
0,AK_Juneau_0.json,../data\AK_Juneau_0.json,8,64,"[last_update_date, tags, permalink, status, li..."
1,AK_Juneau_1.json,../data\AK_Juneau_1.json,7,63,"[primary_photo, last_update_date, source, tags..."
2,AK_Juneau_2.json,../data\AK_Juneau_2.json,6,63,"[primary_photo, last_update_date, source, tags..."
3,AK_Juneau_3.json,../data\AK_Juneau_3.json,5,63,"[primary_photo, last_update_date, source, tags..."
4,AK_Juneau_4.json,../data\AK_Juneau_4.json,4,63,"[primary_photo, last_update_date, source, tags..."
...,...,...,...,...,...
245,WY_Cheyenne_0.json,../data\WY_Cheyenne_0.json,1,0,[]
246,WY_Cheyenne_1.json,../data\WY_Cheyenne_1.json,1,0,[]
247,WY_Cheyenne_2.json,../data\WY_Cheyenne_2.json,1,0,[]
248,WY_Cheyenne_3.json,../data\WY_Cheyenne_3.json,1,0,[]


##### - Comparing, Checking files

In [449]:
# 226 -- just chose one of the files that has an empty 'results' key
weird_file_path = overview.iloc[226]['file_path']

with open(weird_file_path) as f:
  weird_file = json.load(f)

weird_file['data'].keys()

dict_keys(['total', 'count', 'results'])

In [450]:
len(weird_file['data']['results']), len(data_json['data']['results']) # Weird vs Normal File

(0, 8)

In [451]:
weird_files = overview[overview['cols_count'] == 0]

weird_files.shape

(32, 5)

In [452]:
# Create a list of files with an empty 'results' key or has other problems
weird = []
extra_weird = []
  
for file in weird_files['file_path']:
  
  with open(file) as f:
    weird_file = json.load(f)
  
  if len(weird_file['data']['results']) == 0:
    print(f'{file} returns empty \'results\'')
    weird.append(file)
  
  else:
    print(f'{file} has another problem...')
    extra_weird.append(file)

../data\HI_Honolulu_3.json returns empty 'results'
../data\HI_Honolulu_4.json returns empty 'results'
../data\ME_Augusta_0.json returns empty 'results'
../data\ME_Augusta_1.json returns empty 'results'
../data\ME_Augusta_2.json returns empty 'results'
../data\ME_Augusta_3.json returns empty 'results'
../data\ME_Augusta_4.json returns empty 'results'
../data\MS_Jackson_0.json returns empty 'results'
../data\MS_Jackson_1.json returns empty 'results'
../data\MS_Jackson_2.json returns empty 'results'
../data\MS_Jackson_3.json returns empty 'results'
../data\MS_Jackson_4.json returns empty 'results'
../data\ND_Bismarck_2.json returns empty 'results'
../data\ND_Bismarck_3.json returns empty 'results'
../data\ND_Bismarck_4.json returns empty 'results'
../data\NH_Concord_3.json returns empty 'results'
../data\NH_Concord_4.json returns empty 'results'
../data\SD_Pierre_0.json returns empty 'results'
../data\SD_Pierre_1.json returns empty 'results'
../data\SD_Pierre_2.json returns empty 'results

In [453]:
print(f'\n{len(weird)} Weird Files...\n{weird}')
print(f'\n{len(extra_weird)} Extra Weird Files... \n{extra_weird}')


32 Weird Files...
['../data\\HI_Honolulu_3.json', '../data\\HI_Honolulu_4.json', '../data\\ME_Augusta_0.json', '../data\\ME_Augusta_1.json', '../data\\ME_Augusta_2.json', '../data\\ME_Augusta_3.json', '../data\\ME_Augusta_4.json', '../data\\MS_Jackson_0.json', '../data\\MS_Jackson_1.json', '../data\\MS_Jackson_2.json', '../data\\MS_Jackson_3.json', '../data\\MS_Jackson_4.json', '../data\\ND_Bismarck_2.json', '../data\\ND_Bismarck_3.json', '../data\\ND_Bismarck_4.json', '../data\\NH_Concord_3.json', '../data\\NH_Concord_4.json', '../data\\SD_Pierre_0.json', '../data\\SD_Pierre_1.json', '../data\\SD_Pierre_2.json', '../data\\SD_Pierre_3.json', '../data\\SD_Pierre_4.json', '../data\\VT_Montpelier_0.json', '../data\\VT_Montpelier_1.json', '../data\\VT_Montpelier_2.json', '../data\\VT_Montpelier_3.json', '../data\\VT_Montpelier_4.json', '../data\\WY_Cheyenne_0.json', '../data\\WY_Cheyenne_1.json', '../data\\WY_Cheyenne_2.json', '../data\\WY_Cheyenne_3.json', '../data\\WY_Cheyenne_4.json']


In [454]:
# Safe to drop the weird files from the overview dataframe
drop = [i for i in weird_files.index]

overview.drop(drop, inplace=True) # Originally 250, 5

In [455]:
overview.shape

(218, 5)

In [456]:
overview # I will use the file_path column to load the data

,file_name,file_path,rows_count,cols_count,cols_name
0,AK_Juneau_0.json,../data\AK_Juneau_0.json,8,64,"[last_update_date, tags, permalink, status, li..."
1,AK_Juneau_1.json,../data\AK_Juneau_1.json,7,63,"[primary_photo, last_update_date, source, tags..."
2,AK_Juneau_2.json,../data\AK_Juneau_2.json,6,63,"[primary_photo, last_update_date, source, tags..."
3,AK_Juneau_3.json,../data\AK_Juneau_3.json,5,63,"[primary_photo, last_update_date, source, tags..."
4,AK_Juneau_4.json,../data\AK_Juneau_4.json,4,63,"[primary_photo, last_update_date, source, tags..."
...,...,...,...,...,...
240,WV_Charleston_0.json,../data\WV_Charleston_0.json,42,61,"[last_update_date, tags, permalink, status, li..."
241,WV_Charleston_1.json,../data\WV_Charleston_1.json,42,61,"[last_update_date, tags, permalink, status, li..."
242,WV_Charleston_2.json,../data\WV_Charleston_2.json,41,61,"[last_update_date, tags, permalink, status, li..."
243,WV_Charleston_3.json,../data\WV_Charleston_3.json,40,61,"[last_update_date, tags, permalink, status, li..."


##### - Resuming Data Import Procedures

In [457]:
# Testing the custom function `read_json` which takes a file path and returns a DataFrame
read_json(test_dir).head(2) # Works!

,last_update_date,tags,permalink,status,list_date,open_houses,branding,list_price,property_id,photos,...,location.address.state_code,location.address.line,location.street_view_url,location.county.fips_code,location.county.name,primary_photo,source,products,location.address.coordinate,other_listings
0,2023-09-19T20:52:50Z,"[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,None,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...",...,AK,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN
1,None,None,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9424983842,None,...,AK,8477 Thunder Mountain Rd,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN


In [458]:
# loop over all files and put them into a dataframe
combine_all = []

for path in overview['file_path']:
  df = read_json(path)
  combine_all.append(df)
  print(f'{path} added to the list. {len(combine_all)}/{overview.shape[0]} processed...')

print(f'\n{len(combine_all)} files loaded into the list of DataFrames.')

../data\AK_Juneau_0.json added to the list. 1/218 processed...
../data\AK_Juneau_1.json added to the list. 2/218 processed...
../data\AK_Juneau_2.json added to the list. 3/218 processed...
../data\AK_Juneau_3.json added to the list. 4/218 processed...
../data\AK_Juneau_4.json added to the list. 5/218 processed...
../data\AL_Montgomery_0.json added to the list. 6/218 processed...
../data\AL_Montgomery_1.json added to the list. 7/218 processed...
../data\AL_Montgomery_2.json added to the list. 8/218 processed...
../data\AL_Montgomery_3.json added to the list. 9/218 processed...
../data\AL_Montgomery_4.json added to the list. 10/218 processed...
../data\AR_LittleRock_0.json added to the list. 11/218 processed...
../data\AR_LittleRock_1.json added to the list. 12/218 processed...
../data\AR_LittleRock_2.json added to the list. 13/218 processed...
../data\AR_LittleRock_3.json added to the list. 14/218 processed...
../data\AR_LittleRock_4.json added to the list. 15/218 processed...
../data\A

In [459]:
housing_dataset = pd.concat(combine_all, ignore_index=True)

housing_dataset.shape

C:\Users\kadm2\AppData\Local\Temp\ipykernel_16132\3315332010.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  housing_dataset = pd.concat(combine_all, ignore_index=True)


(8159, 67)

#### - Saving outputs...

In [460]:
output_path = '../outputs'

# --- !!! Uncomment only when saving. Run once and put the # back. !!! --- #
# housing_dataset.to_csv(os.path.join(output_path, 'housing_dataset.csv'), index=False)

#### - Load

In [461]:
housing_df = pd.read_csv(os.path.join(output_path, 'housing_dataset.csv'))

housing_df.shape

(8159, 67)

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

#### - Quick look at the data and the values in them.

In [462]:
data = housing_df.copy()

In [463]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8159 entries, 0 to 8158
Data columns (total 67 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   last_update_date                       8125 non-null   object 
 1   tags                                   7638 non-null   object 
 2   permalink                              8159 non-null   object 
 3   status                                 8159 non-null   object 
 4   list_date                              7752 non-null   object 
 5   open_houses                            0 non-null      float64
 6   branding                               8159 non-null   object 
 7   list_price                             7721 non-null   float64
 8   property_id                            8159 non-null   int64  
 9   photos                                 7403 non-null   object 
 10  community                              0 non-null      float64
 11  virt

In [464]:
data.describe()

,open_houses,list_price,property_id,community,listing_id,price_reduced_amount,source.plan_id,description.year_built,description.baths_3qtr,description.sold_price,...,location.address.postal_code,location.address.coordinate.lon,location.address.coordinate.lat,location.county.fips_code,primary_photo,source,products,location.address.coordinate,other_listings,location.county
count,0.0,7.721000e+03,8.159000e+03,0.0,7.752000e+03,2.484000e+03,5.000000e+00,7316.000000,566.000000,6.716000e+03,...,8159.000000,7909.000000,7909.000000,7588.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,4.341582e+05,5.755508e+09,NaN,2.957819e+09,2.442704e+04,4.170007e+11,1968.916074,1.247350,4.126050e+05,...,50946.997181,-92.206522,39.009689,28000.253295,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,5.514925e+05,2.687366e+09,NaN,7.541620e+07,7.162396e+04,0.000000e+00,35.096914,0.463482,6.994308e+05,...,29257.110670,15.888886,4.374553,15586.751739,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000e+00,1.003443e+09,NaN,6.052327e+08,1.000000e+02,4.170007e+11,1828.000000,1.000000,3.080000e+02,...,2111.000000,-157.810583,21.277707,1101.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,2.090000e+05,3.307743e+09,NaN,2.959499e+09,6.000000e+03,4.170007e+11,1950.000000,1.000000,1.910000e+05,...,25314.000000,-104.971611,35.688084,16001.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,3.250000e+05,6.000993e+09,NaN,2.960836e+09,1.010000e+04,4.170007e+11,1975.000000,1.000000,3.140000e+05,...,50310.000000,-89.333131,39.698210,27123.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,4.999000e+05,8.169927e+09,NaN,2.961805e+09,2.000000e+04,4.170007e+11,1997.000000,1.000000,4.700000e+05,...,78739.000000,-78.617690,41.832266,41047.000000,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,1.250000e+07,9.993679e+09,NaN,2.963230e+09,2.015999e+06,4.170007e+11,2024.000000,3.000000,2.706500e+07,...,99801.000000,-71.006343,58.396178,55025.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [465]:
u = data.nunique()

for i in u.index:
  print(f'{u[i]} :: :: :: {i}')

1716 :: :: :: last_update_date
1628 :: :: :: tags
1795 :: :: :: permalink
1 :: :: :: status
1636 :: :: :: list_date
0 :: :: :: open_houses
996 :: :: :: branding
711 :: :: :: list_price
1795 :: :: :: property_id
1627 :: :: :: photos
0 :: :: :: community
290 :: :: :: virtual_tours
1704 :: :: :: listing_id
145 :: :: :: price_reduced_amount
2 :: :: :: matterport
1627 :: :: :: primary_photo.href
1 :: :: :: source.plan_id
393 :: :: :: source.agents
1 :: :: :: source.spec_id
1 :: :: :: source.type
147 :: :: :: description.year_built
3 :: :: :: description.baths_3qtr
127 :: :: :: description.sold_date
738 :: :: :: description.sold_price
8 :: :: :: description.baths_full
0 :: :: :: description.name
5 :: :: :: description.baths_half
743 :: :: :: description.lot_sqft
1119 :: :: :: description.sqft
10 :: :: :: description.baths
2 :: :: :: description.sub_type
0 :: :: :: description.baths_1qtr
9 :: :: :: description.garage
7 :: :: :: description.stories
13 :: :: :: description.beds
11 :: :: :: desc

In [466]:
# Quick look at the columns and the data in it -- Using a custom function, it sorts the df by the number of unique values in each column as well. 
columns_overview = cols_overview(data)

columns_overview.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
66,8159,location.county,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
10,8159,community,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
25,8159,description.name,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
37,8159,flags.is_new_construction,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
38,8159,flags.is_for_rent,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"


##### - Pre-cleaning, Dropping rows.

> Some sales may not actually include the sale price (target).  These rows should be dropped.

> Don't keep the list price because it will be too close to the sale price.

In [467]:
drop_cols = ['list_price',
             'source.type', 
             'source.agents', 
             'last_update_date', 
             'price_reduced_amount',
             'other_listings.rdc',
             'last_update_date',
             'location.street_view_url',
             'matterport',
             'products.brand_name',
             'lead_attributes.show_contact_an_agent',
             'permalink',
             'virtual_tours',
             'location.address.line',
             'description.sub_type',
             'flags.is_new_listing']

# Dropped location.address.line because we don't need it. Lat/Lon is enough.

data.drop(drop_cols, axis=1, inplace=True)
data.shape

(8159, 52)

#### - Missing sold_price

In [468]:
data['description.sold_price'].isnull().sum()

1443

In [469]:
missing_sold_price = data[data['description.sold_price'].isnull()]

missing_sold_price

,tags,status,list_date,open_houses,branding,property_id,photos,community,listing_id,primary_photo.href,...,location.county.fips_code,location.county.name,primary_photo,source,products,location.address.coordinate,other_listings,community.advertisers,community.description.name,location.county
0,"['carport', 'community_outdoor_space', 'cul_de...",sold,2023-06-29T21:16:25.000000Z,NaN,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...",NaN,2.957242e+09,https://ap.rdcpix.com/07097d34c98a59ebb7996889...,...,NaN,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,sold,NaN,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",9424983842,NaN,NaN,NaN,NaN,...,NaN,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,sold,NaN,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",9479068516,NaN,NaN,NaN,NaN,...,NaN,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,sold,NaN,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",9879331943,NaN,NaN,NaN,NaN,...,NaN,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,sold,NaN,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",9521639574,NaN,NaN,NaN,NaN,...,NaN,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7320,"['central_air', 'central_heat', 'community_out...",sold,2023-07-07T02:16:44.000000Z,NaN,"[{'name': 'Dijjit, LC', 'photo': None, 'type':...",1834972490,NaN,NaN,2.957450e+09,NaN,...,49035.0,Salt Lake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7321,"['central_air', 'community_outdoor_space', 'la...",sold,2023-10-19T21:08:10.000000Z,NaN,"[{'name': ""Summit Sotheby's International Real...",2597824245,NaN,NaN,2.960709e+09,NaN,...,49035.0,Salt Lake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7322,"['central_heat', 'community_outdoor_space', 'l...",sold,2023-10-15T00:55:32.000000Z,NaN,"[{'name': 'RED ROCK REAL ESTATE', 'photo': Non...",1641410738,NaN,NaN,2.960567e+09,NaN,...,49035.0,Salt Lake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7323,"['rental_property', 'investment_opportunity', ...",sold,2023-10-11T21:16:02.000000Z,NaN,"[{'name': 'Investment Realty Advisors LLC', 'p...",1479984642,NaN,NaN,2.960458e+09,NaN,...,49035.0,Salt Lake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [470]:
# Drop rows where 'description.sold_price' is NaN
data.dropna(subset=['description.sold_price'], inplace=True)

In [471]:
data.shape

(6716, 52)

#### Dealing with Nulls

In [472]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
51,6716,location.county,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
46,6716,products,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
32,6716,flags.is_pending,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
30,6716,flags.is_contingent,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
29,6716,flags.is_subdivision,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"


##### - 6716 nulls (17 Empty columns)

In [473]:
empties = columns_overview[columns_overview['nulls_count'] == data.shape[0]]

empties.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
51,6716,location.county,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
46,6716,products,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
32,6716,flags.is_pending,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
30,6716,flags.is_contingent,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
29,6716,flags.is_subdivision,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"


In [474]:
is_empty = [i for i in empties['col_name']]

len(is_empty), is_empty

(17,
 ['location.county',
  'products',
  'flags.is_pending',
  'flags.is_contingent',
  'flags.is_subdivision',
  'flags.is_for_rent',
  'flags.is_new_construction',
  'description.baths_1qtr',
  'primary_photo',
  'flags.is_coming_soon',
  'source',
  'description.name',
  'flags.is_plan',
  'other_listings',
  'open_houses',
  'location.address.coordinate',
  'community'])

In [475]:
# Drop the columns that are empty
data.drop(columns=is_empty, inplace=True)

data.shape[1] # From 58

35

In [476]:
# Drop removed columns from columns_overview
columns_overview.drop(index=empties.index, inplace=True)

columns_overview.shape

(35, 7)

In [477]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
34,6711,community.description.name,object,1,"[nan, Woods of Copper Creek]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
33,6711,community.advertisers,object,1,"[nan, [{'office': {'hours': 'Monday - Saturday...","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
8,6711,source.plan_id,float64,1,"[nan, 417000743767.0]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
9,6711,source.spec_id,object,1,"[nan, 365-36546-365750000-0016]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
24,6686,flags.is_foreclosure,object,1,"[nan, True]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"


##### - 6711 Nulls : `['community.description.name', 'source.plan_id', 'source.spec_id', 'community.advertisers']`

In [478]:
remove = columns_overview[columns_overview['nulls_count'] == 6711]['col_name']

remove_cols = [i for i in remove]
remove_cols

['community.description.name',
 'community.advertisers',
 'source.plan_id',
 'source.spec_id']

In [479]:
data.drop(columns=remove_cols, inplace=True)

data.shape[1] # From 41

31

In [480]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
22,6686,flags.is_foreclosure,object,1,"[nan, True]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
9,6343,description.baths_3qtr,float64,2,"[nan, 2.0, 1.0]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
13,4739,description.baths_half,float64,5,"[nan, 1.0, 2.0, 3.0, 4.0, 5.0]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
21,4706,flags.is_price_reduced,object,2,"[nan, False, True]","[nan, False, nan, False, False]","[nan, nan, nan, nan, nan]"
17,3231,description.garage,float64,9,"[2.0, 1.0, nan, 3.0, 4.0, 5.0, 9.0, 8.0, 6.0, ...","[2.0, 1.0, nan, nan, nan]","[1.0, nan, nan, nan, nan]"


##### - 6686 Nulls : `'flags.is_foreclosure'`

In [481]:
# The flags.is_foreclosure may be useful. It may have effect with the house price. Replacing the Nan values with 0 and 1.
data['flags.is_foreclosure'] = data['flags.is_foreclosure'].fillna(0).replace({True: 1})

data['flags.is_foreclosure'].unique()

C:\Users\kadm2\AppData\Local\Temp\ipykernel_16132\318830356.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['flags.is_foreclosure'] = data['flags.is_foreclosure'].fillna(0).replace({True: 1})


array([0, 1], dtype=int64)

##### - 6343 Nulls : `'description.baths_3qtr'`

In [482]:
# Replacing the NaN values with 0
data['description.baths_3qtr'] = data['description.baths_3qtr'].fillna(0.)

data['description.baths_3qtr'].unique()

array([0., 2., 1.])

##### - 4739 Nulls : `'description.baths_half'`

In [483]:
# It's possible to have 0 half baths. Replacing the NaN values with 0
data['description.baths_half'] = data['description.baths_half'].fillna(0.)

data['description.baths_half'].unique(), data['description.baths_half'].isnull().sum()

(array([0., 1., 2., 3., 4., 5.]), 0)

##### - 4706 Nulls : `['flags.is_price_reduced']`

In [484]:
# These columns appear to be related.
compare = data[~data['flags.is_price_reduced'].isnull()]

compare['flags.is_price_reduced'].shape

(2010,)

In [485]:
# Null values + Values that are not null should equal the total number of rows in the dataset.
4706 + 2010 == 6716 # All the nulls are in the same rows. Replacing the NaN values with 0 on 'price_reduced_amount' and False on 'flags.is_price_reduced'

True

In [486]:
# Replacing the NaN values with False
data['flags.is_price_reduced'] = data['flags.is_price_reduced'].fillna(False)

C:\Users\kadm2\AppData\Local\Temp\ipykernel_16132\2118188274.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['flags.is_price_reduced'] = data['flags.is_price_reduced'].fillna(False)


In [487]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
17,3231,description.garage,float64,9,"[2.0, 1.0, nan, 3.0, 4.0, 5.0, 9.0, 8.0, 6.0, ...","[2.0, 1.0, nan, nan, nan]","[1.0, nan, nan, nan, nan]"
18,1617,description.stories,float64,7,"[1.0, 2.0, nan, 3.0, 10.0, 6.0, 8.0, 4.0]","[1.0, 2.0, 1.0, 1.0, 1.0]","[2.0, nan, nan, nan, nan]"
14,922,description.lot_sqft,float64,609,"[11761.0, 6534.0, 17424.0, 9712.0, 10890.0, 12...","[11761.0, 6534.0, 17424.0, 9712.0, 10890.0]","[4792.0, 7841.0, 65340.0, nan, 52272.0]"
12,677,description.baths_full,float64,7,"[2.0, nan, 1.0, 3.0, 4.0, 5.0, 6.0, 7.0]","[2.0, 2.0, 2.0, 2.0, 2.0]","[1.0, 1.0, 1.0, nan, 2.0]"
8,675,description.year_built,float64,144,"[1998.0, 1945.0, 1969.0, 1955.0, 1984.0, 1999....","[1998.0, 1945.0, 1969.0, 1955.0, 1984.0]","[1910.0, nan, nan, nan, 1950.0]"


##### - 3231 Nulls : `'description.garage'`

In [488]:
# # We can assume Nan means no garage. Replacing the NaN values with 0
data['description.garage'] = data['description.garage'].fillna(0.)

##### - 1617 Nulls : `'description.stories'`

In [489]:
data['description.stories'].unique() # No 0 stories. Replacing the NaN values with 0

array([ 1.,  2., nan,  3., 10.,  6.,  8.,  4.])

In [490]:
data['description.stories'] = data['description.stories'].fillna(0.)

data['description.stories'].unique(), data['description.stories'].isnull().sum()

(array([ 1.,  2.,  0.,  3., 10.,  6.,  8.,  4.]), 0)

##### - 1168 & 655 Nulls : `['description.lot_sqft', 'description.sqft']`

In [491]:
# Columns appear to be related, may be redundant.
lot_sizes = data[data[['description.lot_sqft', 'description.sqft']].isnull().any(axis=1)]

lot_sizes[['description.lot_sqft', 'description.sqft']]

,description.lot_sqft,description.sqft
40,3397680.0,NaN
81,3397680.0,NaN
121,3397680.0,NaN
162,3397680.0,NaN
203,3397680.0,NaN
...,...,...
8147,18295.0,NaN
8150,8276.0,NaN
8151,NaN,771.0
8152,NaN,1664.0


In [492]:
lot_sizes.loc[:, 'check_property.sqft'] = lot_sizes['description.lot_sqft'] == lot_sizes['description.sqft']

lot_sizes[lot_sizes['check_property.sqft']==True]

C:\Users\kadm2\AppData\Local\Temp\ipykernel_16132\197350453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lot_sizes.loc[:, 'check_property.sqft'] = lot_sizes['description.lot_sqft'] == lot_sizes['description.sqft']


,tags,status,list_date,branding,property_id,photos,listing_id,primary_photo.href,description.year_built,description.baths_3qtr,...,flags.is_foreclosure,location.address.postal_code,location.address.state,location.address.coordinate.lon,location.address.coordinate.lat,location.address.city,location.address.state_code,location.county.fips_code,location.county.name,check_property.sqft


In [493]:
# Merge 'description.lot_sqft' and 'description.sqft' into a new column 'property.sqft'
data['property.sqft'] = data['description.lot_sqft'].combine_first(data['description.sqft'])

data['property.sqft'].isnull().sum()

291

In [494]:
drop_cols = ['description.lot_sqft', 'description.sqft']

data.drop(columns=drop_cols, inplace=True)

##### - 291 Nulls : `'property.sqft'`

In [495]:
leftover_missing_sqft = data[data['property.sqft'].isnull()]

# Drop rows where 'property.sqft' is NaN
data.dropna(subset=['property.sqft'], inplace=True)

##### - 677 Nulls : `'description.baths_full'`

In [496]:
# Replacing the NaN values with 0
data['description.baths_full'] = data['description.baths_full'].fillna(0.)

In [497]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head(10)

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
8,463,description.year_built,float64,144,"[1998.0, 1945.0, 1969.0, 1955.0, 1984.0, 1999....","[1998.0, 1945.0, 1969.0, 1955.0, 1984.0]","[nan, 1910.0, nan, nan, 1950.0]"
27,404,location.county.fips_code,float64,54,"[1101.0, 5119.0, nan, 5125.0, 4013.0, 6067.0, ...","[1101.0, 1101.0, 1101.0, 1101.0, 1101.0]","[54039.0, 54039.0, 54039.0, 54039.0, 54039.0]"
17,321,description.beds,float64,13,"[3.0, 4.0, nan, 2.0, 5.0, 1.0, 6.0, 12.0, 7.0,...","[3.0, 4.0, 3.0, 3.0, 3.0]","[5.0, 3.0, 3.0, 3.0, 3.0]"
5,286,photos,object,1349,"[nan, [{'tags': [{'label': 'house_view', 'prob...","[nan, [{'tags': [{'label': 'house_view', 'prob...","[[{'tags': [{'label': 'house_view', 'probabili..."
7,286,primary_photo.href,object,1349,"[nan, https://ap.rdcpix.com/a5fe1391acf35d43ee...","[nan, https://ap.rdcpix.com/a5fe1391acf35d43ee...",[https://ap.rdcpix.com/43df1ac3415036d0fdba271...
0,189,tags,object,1339,"[['central_air', 'central_heat', 'community_ou...","[['central_air', 'central_heat', 'community_ou...","[['basement', 'garage_1_or_more', 'garage_2_or..."
6,125,listing_id,float64,1385,"[622475855.0, 2961522977.0, 619793175.0, 29573...","[622475855.0, 2961522977.0, 619793175.0, 29573...","[2954540283.0, 2954257214.0, 2956178043.0, 295..."
2,125,list_date,object,1338,"[2023-12-19T18:08:22.000000Z, 2023-11-10T14:20...","[2023-12-19T18:08:22.000000Z, 2023-11-10T14:20...","[2023-04-13T18:27:22Z, 2023-04-05T19:28:04Z, 2..."
14,120,description.baths,float64,10,"[2.0, 3.0, 0.0, 1.0, 4.0, 5.0, nan, 6.0, 7.0, ...","[2.0, 2.0, 2.0, 2.0, 2.0]","[2.0, 1.0, 1.0, 1.0, 2.0]"
24,99,location.address.coordinate.lat,float64,1384,"[32.389075, 32.382748, 32.380023, 32.386844, 3...","[32.389075, 32.382748, 32.380023, 32.386844, 3...","[38.366388, 38.341576, 38.377371, 38.338617, 3..."


##### - 675 Nulls : `'description.year_built'`

In [498]:
# data[data['description.year_built'].isnull()]

In [499]:
# I'll fill the values with the median year built in the dataset, based the city.
# Calculate the median year built for each city, excluding NaN values
median_year_built_by_city = data.groupby('location.address.city')['description.year_built'].transform('median')

# Replace NaN values in 'description.year_built' with the median year built for the corresponding city
data['description.year_built'] = data['description.year_built'].fillna(median_year_built_by_city)

In [500]:
data['description.year_built'].isnull().sum()

37

In [501]:
# There are 85 rows with NaN values in 'description.year_built'. I'll fill them with the median year built in the dataset.
# Calculate the overall median year built, excluding NaN values
overall_median_year_built = data['description.year_built'].median()

# Replace remaining NaN values in 'description.year_built' with the overall median year built
data['description.year_built'] = data['description.year_built'].fillna(overall_median_year_built)

In [502]:
data['description.year_built'].isnull().sum()

0

##### - 443 Nulls : `['primary_photo.href', 'photos']`

In [503]:
# These columns appear to be related.
compare = data[~data[['primary_photo.href', 'photos']].isnull().all(axis=1)]

compare[['primary_photo.href', 'photos']].shape

(6139, 2)

In [504]:
443 + 6273 == 6716 # All the nulls are in the same rows. Creating a new column 'has_photos' which will be 0 or 1, and then dropping both columns.

True

In [505]:
data['has_photos'] = data['photos'].notnull().astype(int)

data['has_photos'].unique()

array([0, 1])

In [506]:
data.drop(columns=['primary_photo.href', 'photos'], inplace=True)

##### - 493 Nulls : `'description.beds'`

In [507]:
check_type = data[data['description.beds'].isnull()] # Null values in 'description.beds'
review = check_type[['description.beds', 'description.type']] # Checking the  type of the property

review['description.type'].unique()

array(['land', 'multi_family', 'single_family', 'other', 'mobile'],
      dtype=object)

In [508]:
# Group by type and calculate the median number of beds for each type
median_beds_by_type = data.groupby('description.type')['description.beds'].transform('median')

# Replace NaN values in 'description.beds' with the median number of beds for the corresponding type
data['description.beds'] = data['description.beds'].fillna(median_beds_by_type)

data['description.beds'].isnull().sum()

5

In [509]:
data[data['description.beds'].isnull()]

,tags,status,list_date,branding,property_id,listing_id,description.year_built,description.baths_3qtr,description.sold_date,description.sold_price,...,location.address.postal_code,location.address.state,location.address.coordinate.lon,location.address.coordinate.lat,location.address.city,location.address.state_code,location.county.fips_code,location.county.name,property.sqft,has_photos
2376,NaN,sold,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",9515169963,NaN,1974.0,0.0,2023-07-24,275000.0,...,62703,Illinois,-89.651613,39.78819,Springfield,IL,17167.0,Sangamon,9614.0,0
2412,NaN,sold,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",9515169963,NaN,1974.0,0.0,2023-07-24,275000.0,...,62703,Illinois,-89.651613,39.78819,Springfield,IL,17167.0,Sangamon,9614.0,0
2447,NaN,sold,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",9515169963,NaN,1974.0,0.0,2023-07-24,275000.0,...,62703,Illinois,-89.651613,39.78819,Springfield,IL,17167.0,Sangamon,9614.0,0
2481,NaN,sold,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",9515169963,NaN,1974.0,0.0,2023-07-24,275000.0,...,62703,Illinois,-89.651613,39.78819,Springfield,IL,17167.0,Sangamon,9614.0,0
2514,NaN,sold,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",9515169963,NaN,1974.0,0.0,2023-07-24,275000.0,...,62703,Illinois,-89.651613,39.78819,Springfield,IL,17167.0,Sangamon,9614.0,0


In [510]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head(10)

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
25,404,location.county.fips_code,float64,54,"[1101.0, 5119.0, nan, 5125.0, 4013.0, 6067.0, ...","[1101.0, 1101.0, 1101.0, 1101.0, 1101.0]","[54039.0, 54039.0, 54039.0, 54039.0, 54039.0]"
0,189,tags,object,1339,"[['central_air', 'central_heat', 'community_ou...","[['central_air', 'central_heat', 'community_ou...","[['basement', 'garage_1_or_more', 'garage_2_or..."
2,125,list_date,object,1338,"[2023-12-19T18:08:22.000000Z, 2023-11-10T14:20...","[2023-12-19T18:08:22.000000Z, 2023-11-10T14:20...","[2023-04-13T18:27:22Z, 2023-04-05T19:28:04Z, 2..."
5,125,listing_id,float64,1385,"[622475855.0, 2961522977.0, 619793175.0, 29573...","[622475855.0, 2961522977.0, 619793175.0, 29573...","[2954540283.0, 2954257214.0, 2956178043.0, 295..."
12,120,description.baths,float64,10,"[2.0, 3.0, 0.0, 1.0, 4.0, 5.0, nan, 6.0, 7.0, ...","[2.0, 2.0, 2.0, 2.0, 2.0]","[2.0, 1.0, 1.0, 1.0, 2.0]"
22,99,location.address.coordinate.lat,float64,1384,"[32.389075, 32.382748, 32.380023, 32.386844, 3...","[32.389075, 32.382748, 32.380023, 32.386844, 3...","[38.366388, 38.341576, 38.377371, 38.338617, 3..."
21,99,location.address.coordinate.lon,float64,1383,"[-86.178412, -86.273286, -86.221454, -86.28438...","[-86.178412, -86.273286, -86.221454, -86.28438...","[-81.643303, -81.644994, -81.661662, -81.65988..."
26,10,location.county.name,object,55,"[Montgomery, Pulaski, Saline, Maricopa, Sacram...","[Montgomery, Montgomery, Montgomery, Montgomer...","[Kanawha, Kanawha, Kanawha, Kanawha, Kanawha]"
23,5,location.address.city,object,82,"[Montgomery, Little Rock, Mabelvale, Alexander...","[Montgomery, Montgomery, Montgomery, Montgomer...","[Charleston, Charleston, Charleston, Charlesto..."
15,5,description.beds,float64,13,"[3.0, 4.0, 2.0, 5.0, 1.0, 6.0, 12.0, 7.0, 8.0,...","[3.0, 4.0, 3.0, 3.0, 3.0]","[5.0, 3.0, 3.0, 3.0, 3.0]"


In [511]:
# Needs further investigation

##### - 179 Nulls : `'description.baths'`

In [512]:
# Look into all the columns that have 'baths' in the name.
data.columns[data.columns.str.contains('baths')]

Index(['description.baths_3qtr', 'description.baths_full',
       'description.baths_half', 'description.baths'],
      dtype='object')

In [513]:
data[['description.baths_3qtr', 'description.baths_full', 'description.baths_half', 'description.baths']].describe()

,description.baths_3qtr,description.baths_full,description.baths_half,description.baths
count,6425.000000,6425.000000,6425.000000,6305.000000
mean,0.073774,1.741323,0.332607,2.185408
std,0.315891,0.947738,0.540141,1.115353
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,2.000000
50%,0.000000,2.000000,0.000000,2.000000
75%,0.000000,2.000000,1.000000,3.000000
max,2.000000,7.000000,5.000000,9.000000


In [514]:
# Redundant columns. It looks like 'description.baths' is the sum of 'description.baths_3qtr', 'description.baths_full', and 'description.baths_half'. Adding all 3 to fill the NaN values in 'description.baths'.
data['description.baths'] = data['description.baths'].fillna(data['description.baths_3qtr'] + data['description.baths_full'] + data['description.baths_half'])

data['description.baths'].isnull().sum()

0

##### - 125 Nulls : `'list_date'`

In [515]:
data['list_date'].isnull().sum()

125

In [516]:
# Converting list_date to datetime
data['list_date'] = pd.to_datetime(data['list_date'], errors='coerce')

In [517]:
# FIlling null values with median list_date in the city.
mean_list_date_by_city = data.groupby('location.address.city')['list_date'].transform('mean')

data['list_date'] = data['list_date'].fillna(mean_list_date_by_city)

data['list_date'].isnull().sum()

59

In [518]:
# Filling the rest of the null values with the median list_date in the dataset.
overall_mean_list_date = data['list_date'].mean()

data['list_date'] = data['list_date'].fillna(overall_mean_list_date)

data['list_date'].isnull().sum()

0

##### - 34 Nulls : `'description.type'`

In [519]:
data['description.type'].unique()

array(['single_family', 'land', 'townhomes', 'multi_family', 'apartment',
       'mobile', 'condos', 'other', 'duplex_triplex',
       'condo_townhome_rowhome_coop', 'condo'], dtype=object)

In [520]:
# Replacing the NaN values with other
data['description.type'] = data['description.type'].fillna('other')

data['description.type'].unique(), data['description.type'].isnull().sum()

(array(['single_family', 'land', 'townhomes', 'multi_family', 'apartment',
        'mobile', 'condos', 'other', 'duplex_triplex',
        'condo_townhome_rowhome_coop', 'condo'], dtype=object),
 0)

##### - Address-related

10 Nulls : `'location.county.name'`

In [521]:
# Check with other columns.
county = data[data[['location.county.name', 'location.address.state']].isnull().any(axis=1)]

# county[['location.address.city', 'location.county.name', 'location.address.state']]

In [522]:
# Calculate the mode city for each group of county name and state
median_county_by_group = data.groupby(['location.address.city', 'location.address.state'])['location.county.name'].transform(lambda x: x.mode().iloc[0])

# Replace NaN values in 'location.address.city' with the mode city for the corresponding group
data['location.county.name'] = data['location.county.name'].fillna(median_county_by_group)

data['location.county.name'].isnull().sum()

0

5 Nulls : `'location.address.city'`

In [523]:
# Check with other columns.
city = data[data[['location.address.city', 'location.county.name', 'location.address.state']].isnull().any(axis=1)]

# city[['location.address.city', 'location.county.name', 'location.address.state']]

In [524]:
# Calculate the mode city for each group of county name and state
median_city_by_group = data.groupby(['location.county.name', 'location.address.state'])['location.address.city'].transform(lambda x: x.mode().iloc[0])

# Replace NaN values in 'location.address.city' with the mode city for the corresponding group
data['location.address.city'] = data['location.address.city'].fillna(median_city_by_group)

data['location.address.city'].isnull().sum()

0

179 Nulls : `['location.address.coordinate.lon', 'location.address.coordinate.lat']`

In [525]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head(5)

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
25,404,location.county.fips_code,float64,54,"[1101.0, 5119.0, nan, 5125.0, 4013.0, 6067.0, ...","[1101.0, 1101.0, 1101.0, 1101.0, 1101.0]","[54039.0, 54039.0, 54039.0, 54039.0, 54039.0]"
0,189,tags,object,1339,"[['central_air', 'central_heat', 'community_ou...","[['central_air', 'central_heat', 'community_ou...","[['basement', 'garage_1_or_more', 'garage_2_or..."
5,125,listing_id,float64,1385,"[622475855.0, 2961522977.0, 619793175.0, 29573...","[622475855.0, 2961522977.0, 619793175.0, 29573...","[2954540283.0, 2954257214.0, 2956178043.0, 295..."
22,99,location.address.coordinate.lat,float64,1384,"[32.389075, 32.382748, 32.380023, 32.386844, 3...","[32.389075, 32.382748, 32.380023, 32.386844, 3...","[38.366388, 38.341576, 38.377371, 38.338617, 3..."
21,99,location.address.coordinate.lon,float64,1383,"[-86.178412, -86.273286, -86.221454, -86.28438...","[-86.178412, -86.273286, -86.221454, -86.28438...","[-81.643303, -81.644994, -81.661662, -81.65988..."


In [526]:
# Mean lat and lon for each city, state
mean_lat_by_state = data.groupby(['location.address.city', 'location.address.state'])['location.address.coordinate.lat'].transform('mean')
mean_lon_by_state = data.groupby(['location.address.city', 'location.address.state'])['location.address.coordinate.lon'].transform('mean')

# Replace NaN values in 'location.address.coordinate.lat' and 'location.address.coordinate.lon' with the mean lat and lon for the corresponding city
data['location.address.coordinate.lat'] = data['location.address.coordinate.lat'].fillna(mean_lat_by_state)
data['location.address.coordinate.lon'] = data['location.address.coordinate.lon'].fillna(mean_lon_by_state)

data[['location.address.coordinate.lat', 'location.address.coordinate.lon']].isnull().sum()

location.address.coordinate.lat    22
location.address.coordinate.lon    22
dtype: int64

In [527]:
lat = data[data['location.address.coordinate.lat'].isnull()]
lon = data[data['location.address.coordinate.lon'].isnull()]

lat.shape[0], lon.shape[0]

(22, 22)

In [528]:
# Mean lat and lon for each postal code
mean_lat_by_pc = data.groupby('location.address.postal_code')['location.address.coordinate.lat'].transform('mean')
mean_lon_by_pc = data.groupby('location.address.postal_code')['location.address.coordinate.lon'].transform('mean')

# Replace NaN values in 'location.address.coordinate.lat' and 'location.address.coordinate.lon' with the mean lat and lon for the corresponding city
data['location.address.coordinate.lat'] = data['location.address.coordinate.lat'].fillna(mean_lat_by_pc)
data['location.address.coordinate.lon'] = data['location.address.coordinate.lon'].fillna(mean_lon_by_pc)

data[['location.address.coordinate.lat', 'location.address.coordinate.lon']].isnull().sum()

location.address.coordinate.lat    14
location.address.coordinate.lon    14
dtype: int64

In [529]:
# Median lat and lon for each state for the rest of the NaN values
median_lat_by_state = data.groupby('location.address.state')['location.address.coordinate.lat'].transform('median')
median_lon_by_state = data.groupby('location.address.state')['location.address.coordinate.lon'].transform('median')

# Replace NaN values in 'location.address.coordinate.lat' and 'location.address.coordinate.lon' with the median lat and lon for the corresponding city
data['location.address.coordinate.lat'] = data['location.address.coordinate.lat'].fillna(median_lat_by_state)
data['location.address.coordinate.lon'] = data['location.address.coordinate.lon'].fillna(median_lon_by_state)

data['location.address.coordinate.lat'].isnull().sum(), data['location.address.coordinate.lon'].isnull().sum()

(0, 0)

##### - 484 Nulls : `'location.county.fips_code'` -- *Look again*

In [530]:
# data[['location.county.fips_code', 'location.county.name', 'location.address.city', 'location.address.state']]
# Some of the missing values in 'location.county.fips_code' can be filled by looking at the 'location.county.name' and 'location.address.city' columns where it matches.

In [531]:
fips_code_missing = data[data['location.county.fips_code'].isnull()]

fips_code_missing[['location.county.fips_code', 'location.county.name', 'location.address.city', 'location.address.state']].head()

,location.county.fips_code,location.county.name,location.address.city,location.address.state
251,NaN,Pulaski,Little Rock,Arkansas
262,NaN,Pulaski,Little Rock,Arkansas
292,NaN,Pulaski,Little Rock,Arkansas
303,NaN,Pulaski,Little Rock,Arkansas
333,NaN,Pulaski,Little Rock,Arkansas


In [532]:
# Calculate the median fips_code for each group of county name, city, and state
median_fips_by_group = data.groupby(['location.county.name', 'location.address.city', 'location.address.state'])['location.county.fips_code'].transform('median')

# Replace NaN values in 'location.county.fips_code' with the median fips_code for the corresponding group
data['location.county.fips_code'] = data['location.county.fips_code'].fillna(median_fips_by_group)

data['location.county.fips_code'].isnull().sum()

327

In [533]:
view = data[['location.county.fips_code', 'location.county.name', 'location.address.city', 'location.address.state']]

view = view[view['location.county.fips_code'].isnull()]

view

,location.county.fips_code,location.county.name,location.address.city,location.address.state
1961,NaN,Boone,Boone,Iowa
2003,NaN,Boone,Boone,Iowa
2009,NaN,Boone,Boone,Iowa
2043,NaN,Boone,Boone,Iowa
2050,NaN,Boone,Boone,Iowa
...,...,...,...,...
7979,NaN,Kanawha,Charlton Heights,West Virginia
8020,NaN,Kanawha,Charlton Heights,West Virginia
8061,NaN,Kanawha,Charlton Heights,West Virginia
8101,NaN,Kanawha,Charlton Heights,West Virginia


In [534]:
view['location.county.name'].unique()

array(['Boone', 'Carson City', 'Hampton', 'Richmond City', 'Kanawha'],
      dtype=object)

In [535]:
# Missing fips code by location
county_boone = data[data['location.county.name'] == 'Boone']
county_carson = data[data['location.county.name'] == 'Carson City']
county_hampton = data[data['location.county.name'] == 'Hampton']
county_richmond = data[data['location.county.name'] == 'Richmond City']
county_kanawha = data[data['location.county.name'] == 'Kanawha']

# For indexing purposes
Boone = (data['location.county.name'] == 'Boone')
Carson_City = (data['location.county.name'] == 'Carson City')
Hampton = (data['location.county.name'] == 'Hampton')
Richmond_City = (data['location.county.name'] == 'Richmond City')
Kanawha = (data['location.county.name'] == 'Kanawha')

In [536]:
# FIPS Codes are unique to each county and is publicly available data. I'm filling the missing values manually.
# 19015 = Boone County, IA
data.loc[Boone, 'location.county.fips_code'] = data.loc[Boone, 'location.county.fips_code'].fillna(19015)

# 32510 = Carson City, NV
data.loc[Carson_City, 'location.county.fips_code'] = data.loc[Carson_City, 'location.county.fips_code'].fillna(32510)

# 45049 = Hampton, SC
data.loc[Hampton, 'location.county.fips_code'] = data.loc[Hampton, 'location.county.fips_code'].fillna(45049)

# 51760 = Richmond City, VA
data.loc[Richmond_City, 'location.county.fips_code'] = data.loc[Richmond_City, 'location.county.fips_code'].fillna(51159)

# 54039 = Kanawha, WV
data.loc[Kanawha, 'location.county.fips_code'] = data.loc[Kanawha, 'location.county.fips_code'].fillna(54039)

In [537]:
data['location.county.fips_code'].isnull().sum()

0

In [538]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head(5)

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
0,189,tags,object,1339,"[['central_air', 'central_heat', 'community_ou...","[['central_air', 'central_heat', 'community_ou...","[['basement', 'garage_1_or_more', 'garage_2_or..."
5,125,listing_id,float64,1385,"[622475855.0, 2961522977.0, 619793175.0, 29573...","[622475855.0, 2961522977.0, 619793175.0, 29573...","[2954540283.0, 2954257214.0, 2956178043.0, 295..."
15,5,description.beds,float64,13,"[3.0, 4.0, 2.0, 5.0, 1.0, 6.0, 12.0, 7.0, 8.0,...","[3.0, 4.0, 3.0, 3.0, 3.0]","[5.0, 3.0, 3.0, 3.0, 3.0]"
27,0,property.sqft,float64,734,"[11761.0, 6534.0, 17424.0, 9712.0, 10890.0, 12...","[11761.0, 6534.0, 17424.0, 9712.0, 10890.0]","[11326.0, 4792.0, 7841.0, 65340.0, 52272.0]"
26,0,location.county.name,object,55,"[Montgomery, Pulaski, Saline, Maricopa, Sacram...","[Montgomery, Montgomery, Montgomery, Montgomer...","[Kanawha, Kanawha, Kanawha, Kanawha, Kanawha]"


##### - 386 Nulls : `'tags'` -- *Gets its own section further down*

##### Dealing with Duplicates

> Dropping duplicates now. Needed the rest of the data to fill in NaNs.

In [540]:
# Check the duplicates
duplicates = data[data.duplicated()]

duplicates.shape # That's a lot of duplicates...

(5013, 29)

In [541]:
# Dropping duplicates
data.drop_duplicates(inplace=True)

data.shape

(1412, 29)

##### Final touch-ups and saving the data.

In [550]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head(5)

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
0,41,tags,object,1339,"[['central_air', 'central_heat', 'community_ou...","[['central_air', 'central_heat', 'community_ou...","[['basement', 'garage_1_or_more', 'garage_2_or..."
14,1,description.beds,float64,13,"[3.0, 4.0, 2.0, 5.0, 1.0, 6.0, 12.0, 7.0, 8.0,...","[3.0, 4.0, 3.0, 3.0, 3.0]","[5.0, 3.0, 3.0, 3.0, 3.0]"
2,0,list_date,"datetime64[ns, UTC]",1289,"[2023-12-19 18:08:22+00:00, 2023-11-10 14:20:3...","[2023-12-19 18:08:22+00:00, 2023-11-10 14:20:3...","[2023-09-04 08:07:04.900000+00:00, 2023-09-04 ..."
16,0,flags.is_price_reduced,bool,2,"[False, True]","[False, False, False, False, False]","[False, False, False, False, False]"
27,0,has_photos,int32,2,"[0, 1]","[0, 1, 0, 1, 1]","[1, 1, 1, 1, 1]"


In [542]:
# Dropping listing_id because it's just an identifier and there's nulls. I'll just generate our own identifier.
data.drop(columns='listing_id', inplace=True)
data['listing_id'] = data.apply(lambda row: generate_id(), axis=1)

In [549]:
# Save the cleaned data
# --- !!! Uncomment only when saving. Run once and put the # back. !!! --- #
data.to_csv(os.path.join(output_path, 'semi-cleaned_housing_dataset.csv'), index=False)

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [544]:
# OHE categorical variables/ tags here
# tags will have to be done manually

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [545]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [546]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [547]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized

In [548]:
processed_path = '../processed'